<h2>Seguimiento a la Consolidación de Información</h2>

In [ ]:
import os
import pandas as pd
from pathlib import Path
import datetime
import shutil

<h3>Parametrización de variables</h3>

In [ ]:

RUTA_XLSX_NUBE = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\4.Seguimiento_Consolidacion\Reporte_Seguimiento_Consolidacion_V1.xlsx"

RUTA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\4.Seguimiento_Consolidacion\Reporte_Seguimiento_Consolidacion_V1.xlsx"
SHEET_NAME = "Seguimiento Consolidación"
RUTA_BASE = r"E:\Insumos PCCasa"

columnas = ['ruta_archivo','nomArchivo','extensionArchivo','pesoArchivoMbs','fechaCreacionArchivo','fuenteArchivo']
ruta_salida_csv = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\4.Seguimiento_Consolidacion"
nombre_csv = "seguimiento_consolidacion.csv"
extensiones = ['.shp','.xls','.xlsx','.csv','.txt','.doc','.docx','.dwg','.dxf','.pdf', '.geojson', '.json', '.xtf', '.ili',
    '.SHP','.XLS','.XLSX','.CSV','.TXT','.DOC','.DOCX','.DWG','.DXF','.PDF', '.GEOJSON', '.JSON', '.XTF', '.ILI']

reemplazo = {'ruta_archivo':'Ruta Archivo',
    'nomArchivo':'Nombre Archivo',
    'extensionArchivo':'Extensión Archivo',
    'pesoArchivoMbs':'Peso Archivo [Mbs]',
    'fechaCreacionArchivo':'Fecha Creación Archivo',
    'fuenteArchivo':'Entidad\Agencia Fuente',
    'tipoInsumo':'Tipo Insumo'}

COLUMNAS = ['Ruta Archivo',
    'Nombre Archivo',
    'Extensión Archivo',
    'Peso Archivo [Mbs]',
    'Fecha Creación Archivo',
    'Entidad\Agencia Fuente',
    'Tipo Insumo',
    'Consolidado',
    'Fecha Consolidación',
    'Identificador Insumo Inventario',
    'Nombre Archivo o Daset Consolidado En BD',
    'Ruta Consolidacion Insumo Documental']

<h3>Definición de Funciones</h3>

In [ ]:
def ftipoinsumo(a):

    # todo Documental
    a.loc[(a['tipoInsumo'].isnull()) & 
        ((a['extensionArchivo'] == '.pdf') |
        (a['extensionArchivo'] == '.PDF') |
        (a['extensionArchivo'] == '.doc') |
        (a['extensionArchivo'] == '.DOC') |
        (a['extensionArchivo'] == '.docx') |
        (a['extensionArchivo'] == '.DOCX')), 'tipoInsumo'] = 'Documental'

    # todo Geográfica
    a.loc[(a['tipoInsumo'].isnull()) & 
        ((a['extensionArchivo'] == '.gdb') |
        (a['extensionArchivo'] == '.GDB') |
        (a['extensionArchivo'] == '.shp') |
        (a['extensionArchivo'] == '.SHP') |
        (a['extensionArchivo'] == '.geojson') |
        (a['extensionArchivo'] == '.GEOJSON')), 'tipoInsumo'] = 'Geografica'

    # todo Alfanumérica
    a.loc[(a['tipoInsumo'].isnull()) & 
        ((a['extensionArchivo'] == '.xlsx') |
        (a['extensionArchivo'] == '.XLSX') |
        (a['extensionArchivo'] == '.xls') |
        (a['extensionArchivo'] == '.XLS') |
        (a['extensionArchivo'] == '.json') |
        (a['extensionArchivo'] == '.JSON') |
        (a['extensionArchivo'] == '.xtf') |
        (a['extensionArchivo'] == '.XTF') |
        (a['extensionArchivo'] == '.ili') |
        (a['extensionArchivo'] == '.ILI') |
        (a['extensionArchivo'] == '.txt') |
        (a['extensionArchivo'] == '.TXT') |
        (a['extensionArchivo'] == '.csv') |
        (a['extensionArchivo'] == '.CSV') |
        (a['extensionArchivo'] == '.dwg') |
        (a['extensionArchivo'] == '.DWG') |
        (a['extensionArchivo'] == '.dxf') |
        (a['extensionArchivo'] == '.DXF')), 'tipoInsumo'] = 'Geografica'
    
def eliminar_raiz(ruta):
    ruta_completa = Path(ruta)
    partes = ruta_completa.parts
    ruta_sin_raiz = Path(*partes[1:])
    return str(ruta_sin_raiz)

In [ ]:
rutaBorrar = r"E:\Insumos PCCasa\IGAC - Direccion Territorial Atlantico\634\634 - Sabanagrande_Atlantico.txt"
ruta = eliminar_raiz(rutaBorrar)
ruta

<h3>Copia de archivo de seguimiento de Nube o Local</h3>

In [ ]:
shutil.copy(RUTA_XLSX_NUBE, RUTA_XLSX)
print(f"Se copia archivo de ruta {RUTA_XLSX_NUBE} a ruta {RUTA_XLSX}")

<h3>DF con la base de seguimiento a la consolidación</h3>

In [ ]:
df_data_xlsx = pd.read_excel(RUTA_XLSX, sheet_name=SHEET_NAME)

<h3>Indexación de fuente diferentes a las bases de datos geográficas</h3>

In [ ]:
ruta_archivo = []
nomArchivo = []
extensionArchivo = []
pesoArchivoMbs = []
fechaCreacionArchivo = []
fuenteArchivo = []

for ruta, directorio, archivo in os.walk(RUTA_BASE):
    for nombre_archivo in archivo:        
        ruta_archivos = os.path.join(ruta, nombre_archivo)
        rArchivos = Path(ruta_archivos)
        extensionArchivos = rArchivos.suffix
        for extBase in extensiones:
            if extBase == rArchivos.suffix:
                ruta_archivo.append(ruta_archivos)
                nomArchivo.append(rArchivos.stem)
                fuenteArchivo.append(ruta_archivos.split('\\')[2])                
                extensionArchivo.append(rArchivos.suffix)
                pesoArchivoMbs.append((os.path.getsize(rArchivos))/1000000)
                # ** DE UT a Formato Fecha
                marca_tiempo_creacion = os.path.getctime(rArchivos)
                fecha_creacion = datetime.datetime.fromtimestamp(marca_tiempo_creacion)
                fecha_formateada = fecha_creacion.strftime('%Y-%m-%d')
                fechaCreacionArchivo.append(fecha_formateada)
                print("Se documenta {0}".format(rArchivos.name))

<h3>Indexación de las bases de datos geográficas</h3>

In [ ]:
for ruta, directorios, archivo in os.walk(RUTA_BASE):
    for nombre_directorio in directorios:        
        ruta_directorios = os.path.join(ruta, nombre_directorio)
        rDirectorios = Path(ruta_directorios)
        if rDirectorios.suffix == '.gdb':
                ruta_archivo.append(ruta_directorios)                
                nomArchivo.append(rDirectorios.stem)
                fuenteArchivo.append(ruta_directorios.split('\\')[2])
                extensionArchivo.append(rDirectorios.suffix)
                pesoArchivoMbs.append((os.path.getsize(rDirectorios))/1000000)
                # ** DE UT a Formato Fecha
                marca_tiempo_creacion = os.path.getctime(rDirectorios)
                fecha_creacion = datetime.datetime.fromtimestamp(marca_tiempo_creacion)
                fecha_formateada = fecha_creacion.strftime('%Y-%m-%d')
                fechaCreacionArchivo.append(fecha_formateada)
                print("Se documenta {0}".format(rDirectorios.name))


<h3>Unificación de Consolidación Alefanumérica, Geográfica y Documental</h3>

In [ ]:

lista_archivos_recibidos = list(zip(ruta_archivo, nomArchivo, extensionArchivo, pesoArchivoMbs, fechaCreacionArchivo, fuenteArchivo))
df_lista_archivos_recibidos = pd.DataFrame(lista_archivos_recibidos, columns = columnas)

df_lista_archivos_recibidos["tipoInsumo"] = None

# TODO: Inserción del Tipo de Insumo por medio de la función creada
ftipoinsumo(df_lista_archivos_recibidos)

In [ ]:

# ** Se ejecuta para remover las raices de los archivos
# df_data_xlsx['Ruta Archivo'] = df_data_xlsx['Ruta Archivo'].apply(eliminar_raiz)
df_lista_archivos_recibidos['ruta_archivo'] = df_lista_archivos_recibidos['ruta_archivo'].apply(eliminar_raiz)

<h3>Identificación de nuevos insumos frente al documento base de Seguimiento</h3>

In [ ]:

# ** Terrenos Gestionados SIG Espacializados
df_rastreo_vs_base = pd.merge(df_data_xlsx, df_lista_archivos_recibidos, left_on='Ruta Archivo', right_on='ruta_archivo', how='outer')
df_rastreo_vs_base_outer = df_rastreo_vs_base.loc[df_rastreo_vs_base['Ruta Archivo'].isnull()]
df_nuevos_insumos = pd.DataFrame(df_rastreo_vs_base_outer, columns=columnas)

df_nuevos_insumos["tipoInsumo"] = None

# TODO: Inserción del Tipo de Insumo por medio de la función creada
ftipoinsumo(df_nuevos_insumos)

<h3>Reemplazo en el nombre de las columnas</h3>

In [ ]:
df_nuevos_insumos_estandarizado = df_nuevos_insumos.rename(columns=reemplazo)

<h3>Unificación de los nuevos insumos dentro del documento de seguimiento</h3>

In [ ]:
df_nuevos_insumos_estandarizado_tipo_insumo = pd.concat([df_data_xlsx, df_nuevos_insumos_estandarizado] , ignore_index=True)

df_seguimiento_consolidacion = pd.DataFrame(df_nuevos_insumos_estandarizado_tipo_insumo, columns=COLUMNAS)

<h3>Parametrización de lo NO consolidado</h3>

In [ ]:
df_seguimiento_consolidacion.loc[df_seguimiento_consolidacion['Consolidado'].isnull(),'Consolidado'] = 'No'

<h3>Generación de CSV para reemplazar el informe de Seguimiento</h3>

In [ ]:
df_seguimiento_consolidacion.to_csv(os.path.join(ruta_salida_csv, nombre_csv))
print(f"Se genera CSV: {nombre_csv}")

<h3>Estadísticos</h3>

In [ ]:
print(f"Total de registros, XLSX Base: {df_data_xlsx.shape[0]}")
print(f"Total de actualizados: {df_rastreo_vs_base_outer.shape[0]}")
print(f"Total de registros, Seguimiento Consolidado: {df_seguimiento_consolidacion.shape[0]}")